# To do:

 - Figure out how to signal injuries

In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-03


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [5]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [6]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['Season', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

# Main Model

In [10]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [11]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [12]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

107 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
90,NYK,Karl-Anthony Towns,C,PHI,28.411846,31.899277,21.5,38.096642
51,NYK,Mikal Bridges,SF,PHI,35.643612,36.355669,16.5,8.111785
33,NYK,Jordan Clarkson,SG,PHI,18.357082,22.636298,7.5,11.240040
18,NYK,Tyler Kolek,PG,PHI,19.824503,21.131792,5.5,7.116846
29,NYK,Miles McBride,SG,PHI,29.077999,27.728961,12.5,13.120092
75,NYK,OG Anunoby,PF,PHI,35.678345,34.496019,17.5,17.978960
6,NYK,Jalen Brunson,PG,PHI,36.006405,36.650040,29.5,29.207666


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
92,LAC,Ivica Zubac,C,BOS,25.832159,24.283150,11.5,0.950247
88,LAC,Nicolas Batum,PF,BOS,25.463804,22.672493,4.5,8.324839
46,LAC,Kawhi Leonard,SF,BOS,37.583946,32.886784,28.5,31.653894
5,LAC,James Harden,PG,BOS,36.518173,32.909137,24.5,27.394623
82,LAC,John Collins,PF,BOS,25.783493,26.071772,10.5,12.690208
105,LAC,Brook Lopez,C,BOS,21.439234,20.112105,6.5,8.471767
15,LAC,Kris Dunn,PG,BOS,28.291254,27.044938,6.5,7.139578
59,LAC,Derrick Jones Jr.,SF,BOS,21.744738,15.199693,8.5,8.783381


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
66,DAL,Anthony Davis,PF,HOU,34.038921,31.162738,21.5,30.039146
54,DAL,Naji Marshall,SF,HOU,28.761560,31.297777,9.5,14.265941
35,DAL,Max Christie,SG,HOU,29.052824,26.891979,9.5,13.059523
57,DAL,Klay Thompson,SF,HOU,18.658770,21.560114,8.5,10.407587
13,DAL,Cooper Flagg,PG,HOU,37.091766,35.560542,19.5,18.053246
71,DAL,P.J. Washington,PF,HOU,32.275539,31.248331,13.5,12.870639


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
38,TOR,Ja'Kobe Walter,SG,ATL,21.641922,20.411245,5.5,11.377059
48,TOR,Brandon Ingram,SF,ATL,35.587234,34.184127,23.5,19.267429
10,TOR,Immanuel Quickley,PG,ATL,33.627201,32.351594,16.5,19.652010
17,TOR,Jamal Shead,PG,ATL,22.768850,23.241946,7.5,10.456192
52,TOR,RJ Barrett,SF,ATL,26.863726,27.407697,15.5,16.928852
39,TOR,Ochai Agbaji,SG,ATL,17.580442,21.406557,5.5,6.781183
98,TOR,Sandro Mamukelashvili,C,ATL,19.450701,23.777174,10.5,11.350353
86,TOR,Collin Murray-Boyles,PF,ATL,23.070410,18.790290,6.5,7.249655
73,TOR,Scottie Barnes,PF,ATL,35.444263,34.883243,19.5,19.509686


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
96,ATL,Onyeka Okongwu,C,TOR,34.439484,33.443823,17.5,21.657928
16,ATL,Vit Krejci,PG,TOR,26.707254,22.998086,9.5,12.038268
60,ATL,Zaccharie Risacher,SF,TOR,23.109283,22.651906,10.5,8.036184
84,ATL,Asa Newell,PF,TOR,11.743740,13.734236,6.5,5.809936
24,ATL,Nickeil Alexander-Walker,SG,TOR,33.597206,32.162597,20.5,19.811398
34,ATL,Dyson Daniels,SG,TOR,35.571247,33.896677,11.5,11.979294
49,ATL,Jalen Johnson,SF,TOR,36.261528,35.066270,23.5,23.300409


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
45,POR,Deni Avdija,SF,SAS,36.292770,35.302800,26.5,23.604012
20,POR,Shaedon Sharpe,SG,SAS,33.028393,31.405017,24.5,22.691450
62,POR,Kris Murray,SF,SAS,26.109585,27.214075,6.5,7.853286
100,POR,Donovan Clingan,C,SAS,29.295954,27.913218,11.5,12.442021
83,POR,Toumani Camara,PF,SAS,33.614189,32.182790,12.5,12.276896


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
47,HOU,Amen Thompson,SF,DAL,36.934551,34.282750,16.5,18.877586
89,HOU,Alperen Sengun,C,DAL,34.646507,31.741766,20.5,19.333504
9,HOU,Reed Sheppard,PG,DAL,28.971821,26.455751,12.5,13.589486
43,HOU,Kevin Durant,SF,DAL,36.931919,33.506115,24.5,25.559866
72,HOU,Jabari Smith Jr.,PF,DAL,35.127918,32.792640,14.5,13.810349
77,HOU,Tari Eason,PF,DAL,21.624552,19.634111,11.5,11.823900


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
106,BOS,Luka Garza,C,LAC,20.034761,17.606935,7.5,9.676743
21,BOS,Derrick White,SG,LAC,36.739292,34.553576,18.5,20.631426
41,BOS,Jaylen Brown,SF,LAC,34.208282,32.109865,28.5,27.206188
103,BOS,Neemias Queta,C,LAC,26.719072,26.565491,9.5,10.380902
87,BOS,Jordan Walsh,PF,LAC,13.446157,18.955074,4.5,3.976310
14,BOS,Payton Pritchard,PG,LAC,34.122993,33.021851,16.5,16.661606
26,BOS,Anfernee Simons,SG,LAC,26.084888,26.420839,11.5,11.519973


../tables/2025/gmday_preds_PTS.csv saved!


In [14]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

80 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
90,NYK,Karl-Anthony Towns,C,PHI,28.411846,31.899277,36.5,55.712944
51,NYK,Mikal Bridges,SF,PHI,35.643612,36.355669,26.5,22.472406
18,NYK,Tyler Kolek,PG,PHI,19.824503,21.131792,11.5,14.783369
75,NYK,OG Anunoby,PF,PHI,35.678345,34.496019,27.5,25.116663
29,NYK,Miles McBride,SG,PHI,29.077999,27.728961,18.5,19.833128
6,NYK,Jalen Brunson,PG,PHI,36.006405,36.650040,40.5,39.712711


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
4,SAS,Stephon Castle,PG,POR,32.328163,30.632796,33.5,24.670403
2,SAS,De'Aaron Fox,PG,POR,32.536491,30.925064,32.5,27.370874
28,SAS,Dylan Harper,SG,POR,22.118652,20.973669,21.5,16.885052
61,SAS,Julian Champagnie,SF,POR,27.097775,26.210842,20.5,19.337337
58,SAS,Keldon Johnson,SF,POR,23.595665,20.170511,21.5,20.352251


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
35,DAL,Max Christie,SG,HOU,29.052824,26.891979,13.5,20.582405
66,DAL,Anthony Davis,PF,HOU,34.038921,31.162738,34.5,40.307499
13,DAL,Cooper Flagg,PG,HOU,37.091766,35.560542,29.5,32.852634
54,DAL,Naji Marshall,SF,HOU,28.761560,31.297777,15.5,17.793915
71,DAL,P.J. Washington,PF,HOU,32.275539,31.248331,21.5,19.304811


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
56,MIA,Andrew Wiggins,SF,MIN,32.036499,29.900376,22.5,29.318312
55,MIA,Jaime Jaquez Jr.,SF,MIN,31.379396,29.122285,24.5,30.989481
85,MIA,Nikola Jovic,PF,MIN,23.177940,14.184584,17.5,22.472679
25,MIA,Norman Powell,SG,MIN,33.264751,30.163918,30.5,31.569702
12,MIA,Davion Mitchell,PG,MIN,29.313383,27.450117,17.5,16.498808
91,MIA,Bam Adebayo,C,MIN,32.347660,30.256756,28.5,28.493401


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
20,POR,Shaedon Sharpe,SG,SAS,33.028393,31.405017,31.5,27.938684
45,POR,Deni Avdija,SF,SAS,36.292770,35.302800,42.5,40.068214
83,POR,Toumani Camara,PF,SAS,33.614189,32.182790,21.5,19.806179


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
42,CHO,Brandon Miller,SF,CHI,32.673813,33.007659,29.5,32.882801
65,CHO,Miles Bridges,PF,CHI,33.991241,32.410394,28.5,26.469828
44,CHO,Kon Knueppel,SF,CHI,32.169971,32.868275,28.5,27.127329
1,CHO,LaMelo Ball,PG,CHI,29.198853,28.219849,33.5,33.309704


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
103,BOS,Neemias Queta,C,LAC,26.719072,26.565491,19.5,16.277025
41,BOS,Jaylen Brown,SF,LAC,34.208282,32.109865,40.5,43.110752
21,BOS,Derrick White,SG,LAC,36.739292,34.553576,28.5,30.686228
26,BOS,Anfernee Simons,SG,LAC,26.084888,26.420839,16.5,17.812746
14,BOS,Payton Pritchard,PG,LAC,34.122993,33.021851,25.5,25.971193


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
89,HOU,Alperen Sengun,C,DAL,34.646507,31.741766,36.5,34.853584
47,HOU,Amen Thompson,SF,DAL,36.934551,34.282750,29.5,31.083883
9,HOU,Reed Sheppard,PG,DAL,28.971821,26.455751,18.5,20.065248
43,HOU,Kevin Durant,SF,DAL,36.931919,33.506115,35.5,36.412815
72,HOU,Jabari Smith Jr.,PF,DAL,35.127918,32.792640,23.5,24.155556


../tables/2025/gmday_preds_PRA.csv saved!
